In [111]:
from sklearn.decomposition import PCA
import librosa
import numpy as np
import joblib
from scipy.signal import spectrogram
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.fft import rfft
import scipy.io.wavfile as wav
import os

In [123]:

# def extract_passwords_features(path, audio_file=None, sr=None):
#     if audio_file is None or sr is None:
#         audio_file, sr = librosa.load(path)

#     m = np.abs(librosa.stft(y=audio_file, n_fft=512))
#     pca = PCA(n_components=10)  # Reduce to 2 dimensions for visualization
#     # pca_m = pca.fit_transform(m).flatten()
#     m_means = np.mean(m, axis=1).flatten()



#     # Spectrogram
#     f, t, Sxx = spectrogram(audio_file, sr)
#     Sxx_pca = pca.fit_transform(np.abs(Sxx)).flatten() 

    
#     mfccs = librosa.feature.mfcc(y=audio_file, sr=sr, n_mfcc=100)
#     mfccs_means= np.mean(mfccs,axis=1).flatten()

#     combined_features = np.concatenate([m_means, Sxx_pca, mfccs_means])
#     return combined_features


def extract_passwords_features(path, audio_file=None, sr=None):
    if audio_file is None or sr is None:
        audio_file, sr = librosa.load(path)

    m = np.abs(librosa.stft(y=audio_file, n_fft=512))
    pca = PCA(n_components=10)  # Reduce to 2 dimensions for visualization
    pca_m = pca.fit_transform(m).flatten()
    
    mfccs = librosa.feature.mfcc(y=audio_file, sr=sr, n_mfcc=100)
    mfccs_pca= pca.fit_transform(mfccs).flatten()

    # # chroma_stft
    chroma_stft = librosa.feature.chroma_stft(y=audio_file, sr=sr, n_chroma=100)
    chroma_stft_pca = pca.fit_transform(chroma_stft).flatten()

    combined_features = np.concatenate([pca_m,mfccs_pca, chroma_stft_pca])
    return combined_features


def extract_person_features(path, audio_file=None, sr=None):
    if audio_file is None or sr is None:
        audio_file, sr = librosa.load(path)

    pca = PCA(n_components=10)  # Reduce to 2 dimensions for visualization

    # abs of stft
    m = np.abs(librosa.stft(y=audio_file, n_fft=512))
    pca_m = pca.fit_transform(m).flatten()

    # mfccs
    mfccs = librosa.feature.mfcc(y=audio_file, sr=sr, n_mfcc=100)
    mfccs_pca= pca.fit_transform(mfccs).flatten()

    # Spectrogram
    f, t, Sxx = spectrogram(audio_file, sr)
    Sxx_pca = pca.fit_transform(Sxx).flatten()  

    # # chroma_stft
    chroma_stft = librosa.feature.chroma_stft(y=audio_file, sr=sr, n_chroma=100)
    chroma_stft_pca = pca.fit_transform(chroma_stft).flatten()

    combined_features = np.concatenate([pca_m, mfccs_pca, Sxx_pca, chroma_stft_pca])

    return combined_features


In [113]:
class SvcModel:
    def __init__(self, first_folder_path, n=0):
        self.model = SVC(kernel="linear", probability=True, random_state=42,tol=0.01)

        features = []
        labels = []
        for folder1 in os.listdir(first_folder_path):
            for file in os.listdir(first_folder_path + "/" + folder1):
                file = first_folder_path + "/" + folder1 + "/" + file
                if n == 0:
                    features.append(extract_passwords_features(file))
                else:
                    features.append(extract_person_features(file))
                labels.append(folder1)

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(features, labels, test_size=0.2,
                                                                                random_state=42)

    def train(self):
        self.model.fit(self.X_train, self.y_train)

    def predict(self, X_test):
        return self.model.predict(X_test)

    def predict_proba(self, X_test):
        return self.model.predict_proba(X_test)

    def evaluate(self):
        predictions = self.model.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, predictions)
        return accuracy

    def my_classes(self):
        return self.model.classes_


In [114]:

class GbcModel:
    def __init__(self, first_folder_path, n=0):
        self.model = GradientBoostingClassifier(n_estimators=100, random_state=42)

        features = []
        labels = []
        for folder1 in os.listdir(first_folder_path):
            for file in os.listdir(first_folder_path + "/" + folder1):
                file = first_folder_path + "/" + folder1 + "/" + file
                if n == 0:
                    features.append(extract_passwords_features(file))
                else:
                    features.append(extract_person_features(file))
                labels.append(folder1)

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(features, labels, test_size=0.1,
                                                                                random_state=42)

    def train(self):
        self.model.fit(self.X_train, self.y_train)

    def predict(self, X_test):
        return self.model.predict(X_test)

    def predict_proba(self, X_test):
        return self.model.predict_proba(X_test)

    def evaluate(self):
        predictions = self.model.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, predictions)
        return accuracy

    def my_classes(self):
        return self.model.classes_


In [124]:
my_password_model = SvcModel("passwords", 0)
my_password_model.train()
joblib.dump(my_password_model, 'password_model.joblib')
print("password model accuracy: ", my_password_model.evaluate())

password model accuracy:  1.0


In [116]:
my_person_model = SvcModel("people", 1)
my_person_model.train()
joblib.dump(my_person_model, 'person_model.joblib')
print("person model accuracy: ", my_person_model.evaluate())

C:\Users\dell\AppData\Local\Temp\ipykernel_17892\548846539.py:41: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_file, sr = librosa.load(path)
c:\Python311\Lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
C:\Users\dell\AppData\Local\Temp\ipykernel_17892\548846539.py:41: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_file, sr = librosa.load(path)
c:\Python311\Lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
C:\Users\dell\AppData\Local\Temp\ipykernel_17892\548846539.py:41: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_file, sr = librosa.lo

person model accuracy:  1.0


In [117]:
prediction_proba = my_password_model.predict_proba([extract_passwords_features("record.wav")])
classes = my_password_model.model.classes_

print("Class Probabilities:")
print(prediction_proba)
for i, class_label in enumerate(classes):
    print(f"{class_label}: {prediction_proba[0, i]}")

Class Probabilities:
[[0.05323249 0.94283966 0.00392785]]
Grant me access: 0.05323248523720715
Open the door: 0.9428396613094935
Unlock middle gate: 0.003927853453299485


In [118]:
prediction_proba = my_person_model.predict_proba([extract_person_features("record.wav")])
classes = my_person_model.model.classes_
print( my_person_model.predict([extract_person_features("record.wav")])[0])
print("Class Probabilities:")
print(prediction_proba)
for i, class_label in enumerate(classes):
    print(f"{class_label}: {prediction_proba[0, i]}")

Abdallah
Class Probabilities:
[[0.35686021 0.1152248  0.02040147 0.01011902 0.16209072 0.04092638
  0.29437741]]
Abdallah: 0.356860205393809
Amgad: 0.11522480107585235
Amir: 0.020401468777164148
Camellia: 0.010119015712170863
Omar Mo: 0.16209072137411148
Osama: 0.04092638133236221
Ziyad: 0.2943774063345294


In [119]:
# my_Gbc_person_model = GbcModel("people", 1)
# my_Gbc_person_model.train()
# joblib.dump(my_Gbc_person_model, 'gbc_person_model.joblib')
# print("person model accuracy: ", my_Gbc_person_model.evaluate())

In [120]:

# def extract_passwords_features(path, audio_file=None, sr=None):
#     if audio_file is None or sr is None:
#         audio_file, sr = librosa.load(path)

#     m = np.abs(librosa.stft(y=audio_file, n_fft=512))
#     pca = PCA(n_components=10)  # Reduce to 2 dimensions for visualization
#     pca_m = pca.fit_transform(m).flatten()
    
#     mfccs = librosa.feature.mfcc(y=audio_file, sr=sr, n_mfcc=100)
#     mfccs_pca= pca.fit_transform(mfccs).flatten()

#     combined_features = np.concatenate([pca_m,mfccs_pca])
#     return combined_features


# def extract_person_features(path, audio_file=None, sr=None):
#     if audio_file is None or sr is None:
#         audio_file, sr = librosa.load(path)

#     audio_file /= np.max(np.abs(audio_file),axis=0)

#     pca = PCA(n_components=10)  # Reduce to 2 dimensions for visualization

#     # abs of stft
#     m = np.abs(librosa.stft(y=audio_file, n_fft=512))
#     pca_m = pca.fit_transform(m).flatten()

#     # mfccs
#     mfccs = librosa.feature.mfcc(y=audio_file, sr=sr, n_mfcc=100)
#     mfccs_pca= pca.fit_transform(mfccs).flatten()

#     # mel spectrogram
#     # mel_spectrogram = librosa.feature.melspectrogram(y=audio_file, sr=sr, n_mels=100)
#     # mel_spectrogram_pca = pca.fit_transform(mel_spectrogram).flatten()

#     # chroma_stft
#     chroma_stft = librosa.feature.chroma_stft(y=audio_file, sr=sr, n_chroma=100)
#     chroma_stft_pca = pca.fit_transform(chroma_stft).flatten()

#     # crpssing rate
#     zero_crossing_rate = librosa.zero_crossings(y=audio_file).flatten()
#     # zero_crossing_rate_pca = pca.fit_transform(zero_crossing_rate).flatten()

#     # Spectrogram
#     f, t, Sxx = spectrogram(audio_file, sr)
#     Sxx_pca = pca.fit_transform(Sxx).flatten()  


#     # spectral_centroid
#     # spectral_poly_features = librosa.feature.poly_features(y=audio_file, sr=sr, n_fft=512)
#     # pca = PCA(n_components=1) # Reduce to 2 dimensions for visualization
#     # spectral_poly_features_pca = pca.fit_transform(spectral_poly_features).flatten()

#     combined_features = np.concatenate([pca_m, mfccs_pca, Sxx_pca, chroma_stft_pca, zero_crossing_rate])

#     return combined_features
